# Testing Edm4hep.jl

Use the package in the "normal" way, i.e., clone the [Edm4hep.jl](https://github.com/Ananya2003Gupta/Edm4hep.jl).

Setup a project to use it with, e.g.,

```julia
juila> ]

(Edm4hep-testing) pkg> add Revise

(Edm4hep-testing) pkg> dev ../Edm4hep.jl
```

In [1]:
using Revise
using Edm4hep

In [5]:
for name in names(Edm4hep)
    println(name)
end

CaloHitContribution
CaloHitContributionCollection
CalorimeterHit
CalorimeterHitCollection
Cluster
ClusterCollection
Edm4hep
EventHeader
EventHeaderCollection
HitLevelData
Hypothesis
MCParticle
MCParticleCollection
MCRecoCaloAssociation
MCRecoCaloAssociationCollection
MCRecoCaloParticleAssociation
MCRecoCaloParticleAssociationCollection
MCRecoClusterParticleAssociation
MCRecoClusterParticleAssociationCollection
MCRecoParticleAssociation
MCRecoParticleAssociationCollection
MCRecoTrackParticleAssociation
MCRecoTrackParticleAssociationCollection
MCRecoTrackerAssociation
MCRecoTrackerAssociationCollection
MCRecoTrackerHitPlaneAssociation
MCRecoTrackerHitPlaneAssociationCollection
ObjectID
ParticleID
ParticleIDCollection
Quantity
RawCalorimeterHit
RawCalorimeterHitCollection
RawTimeSeries
RawTimeSeriesCollection
RecDqdx
RecDqdxCollection
RecIonizationCluster
RecIonizationClusterCollection
RecoParticleVertexAssociation
RecoParticleVertexAssociationCollection
ReconstructedParticle
Reconstructe

In [6]:
typeof(MCParticle)

typeof(MCParticle) (singleton type of function MCParticle, subtype of Function)

## Basic initialisation of types

Check that basic types can be initialised...

In [7]:
vf=Vector3f(1.0,3.0,6.0)

Edm4hep.Vector3fStruct(1.0f0, 3.0f0, 6.0f0)

In [24]:
wf=Vector3f(3, 6, 2)

Edm4hep.Vector3fStruct(3.0f0, 6.0f0, 2.0f0)

In [10]:
vf.x

1.0f0

## Operations on basic types?

No, these are not supported, they have to be defined:

In [11]:
mag(vf)

UndefVarError: UndefVarError: `mag` not defined

Define a few expected operations...

(Aside - what happens in Edm4hep in Python and C++, are these expected operations there?)

In [14]:
mag(v::Edm4hep.Vector3fStruct) = sqrt(vf.x^2 + vf.y^2 + vf.z^2)

mag (generic function with 1 method)

In [15]:
mag(vf)

6.78233f0

In [17]:
import Base.+

In [18]:
+(v::Edm4hep.Vector3fStruct, w::Edm4hep.Vector3fStruct) = Vector3f(v.x+w.x, v.y+w.y, v.z+w.z)

+ (generic function with 214 methods)

In [25]:
vf+wf

Edm4hep.Vector3fStruct(4.0f0, 9.0f0, 8.0f0)

In [23]:
typeof(w)

Edm4hep.Vector3fStruct

### Natural constructors?

I continue to puzzle over why we wrote constructors with a different underlying datatype...?

Normal style constructors do just work!

In [30]:
Edm4hep.Vector3fStruct() = Edm4hep.Vector3fStruct(0.0f0, 0.0f0, 0.0f0)

In [31]:
origin = Edm4hep.Vector3fStruct()

Edm4hep.Vector3fStruct(0.0f0, 0.0f0, 0.0f0)

In [32]:
v3f_fromint = Edm4hep.Vector3fStruct(3,2,1)

Edm4hep.Vector3fStruct(3.0f0, 2.0f0, 1.0f0)

In [33]:
v3f_fromf64 = Edm4hep.Vector3fStruct(3.1,2.2,1.3)

Edm4hep.Vector3fStruct(3.1f0, 2.2f0, 1.3f0)

## More Sophisticated Datatypes...

Ah, it's not very nice that we don't use named parameters here - they are all positional. We should try and fix that...

Oh, the method signature is horrible here!

In [35]:
electron = ParticleID(type=3, PDG=3)

MethodError: MethodError: no method matching ParticleID(; type::Int64, PDG::Int64)

Closest candidates are:
  ParticleID() got unsupported keyword arguments "type", "PDG"
   @ Edm4hep ~/code/Edm4hep.jl/src/Edm4hep.jl:368
  ParticleID(!Matched::Signed) got unsupported keyword arguments "type", "PDG"
   @ Edm4hep ~/code/Edm4hep.jl/src/Edm4hep.jl:368
  ParticleID(!Matched::Signed, !Matched::Signed) got unsupported keyword arguments "type", "PDG"
   @ Edm4hep ~/code/Edm4hep.jl/src/Edm4hep.jl:368
  ...


In [37]:
electron = ParticleID()

Edm4hep.ParticleIDStruct(0, 0, 0, 0.0f0, Float32[])

In [38]:
electron2 = ParticleID(3, 3, 0, 0.2, [1.2, 3.4])

MethodError: MethodError: no method matching ParticleID(::Int64, ::Int64, ::Int64, ::Float64, ::Vector{Float64})

In [39]:
electron3 = ParticleID(Int32(3), Int32(3))

MethodError: MethodError: no method matching ParticleID(::Int32, ::Int32)

In [46]:
electron2 = ParticleID(Int32(3), Int32(3), Int32(0), 0.2f0, [1.2f0, 3.4f0])

MethodError: MethodError: no method matching ParticleID(::Int32, ::Int32, ::Int32, ::Float32, ::Vector{Float32})

Using the Julia constructors works as expected!

Including changing to the correct internal datatypes by promotion/demotion

In [47]:
delectron = Edm4hep.ParticleIDStruct(Int32(3), Int32(3), Int32(0), 0.2f0, [1.2f0, 3.4f0])

Edm4hep.ParticleIDStruct(3, 3, 0, 0.2f0, Float32[1.2, 3.4])

In [48]:
pelectron = Edm4hep.ParticleIDStruct(3, -3, 12, 0.6, [1.67, 3.22])

Edm4hep.ParticleIDStruct(3, -3, 12, 0.6f0, Float32[1.67, 3.22])